In [1]:
# nothing new to this algorithm, just adding tensorboard inspectors
# to see data. Rewriting a couple of parts to make it reusable,
# for example, the conv and fully connected layers will be created
# via one function

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# we need math for learning rate calculation
import math

# one_hot = True means we represent the data in a vector way, see below
mnist = input_data.read_data_sets('/tmp/data', one_hot=True, reshape=False, validation_size=0)

# random seeding 0 means we will get the same random numbers each time we run the program
tf.set_random_seed(0)

# 99% accuracy baby! Woohooo :D 


TPM_TENSORBOARD_PATH='/tmp/minst'

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# Probability of keeping a node during dropout = 1.0 at test time (no dropout) and 0.75 at training time
pkeep = tf.placeholder(tf.float32, name='dropout_rate')

with tf.name_scope('input'):
    # input X: 28x28 grayscale images, the first dimension (None) will index the images in the mini-batch
    X = tf.placeholder(tf.float32, [None, 28, 28, 1])
    
with tf.name_scope('expectation'):
    # correct answers will go here
    Y_ = tf.placeholder(tf.float32, [None, 10])

# variable learning rate
lr = tf.placeholder(tf.float32)
tf.summary.scalar('learning_rate', lr)

<tf.Tensor 'learning_rate:0' shape=() dtype=string>

In [15]:
# Convolutional layers configuration

# three convolutional layers with their channel counts, and a
# fully connected layer (tha last layer has 10 softmax neurons)
K = 4  # first convolutional layer output depth
L = 8  # second convolutional layer output depth
M = 12  # third convolutional layer
N = 200  # fully connected layer

def register_conv_layer_inspection(input_matrix, size, name):
    net_transposed = input_matrix
    #net_transposed = tf.transpose (net_transposed, [3, 0, 1, 2])
    net_transposed = tf.reduce_mean(net_transposed, 3, keep_dims=True)
    net_transposed = tf.slice(net_transposed, [0,0,0,0], [-1,-1,-1,-1])
    #print ('after transpose', net_transposed)
    #net_transposed = tf.slice(net_transposed[0], [0,0,0], [-1,-1,-1])
    #print ('after slice', net_transposed)
    #net_transposed = tf.reshape(net_transposed, shape=[-1, size, size, 1])
    #print ('after reshape', net_transposed)

    # this will display random 3 filters from the 64 in conv1
    tf.summary.image(
        name=name, 
        tensor=net_transposed,
        max_outputs=3
    )

with tf.name_scope('conv1'):    
    W1 = tf.Variable(tf.truncated_normal([6, 6, 1, K], stddev=0.1))  # 5x5 patch, 1 input channel, K output channels
    B1 = tf.Variable(tf.ones([K])/10)

    stride = 1  # output is 28x28
    Y1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding='SAME') + B1)
    tf.summary.histogram('weights', W1)
    tf.summary.histogram('bias', B1)
    tf.summary.histogram('act', Y1)

    register_conv_layer_inspection(Y1, 28, 'conv1/filters')
    
with tf.name_scope('conv2'):    
    W2 = tf.Variable(tf.truncated_normal([6, 6, K, L], stddev=0.1))
    B2 = tf.Variable(tf.ones([L])/10)
    stride = 2  # output is 14x14
    Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=[1, stride, stride, 1], padding='SAME') + B2)
    
    tf.summary.histogram('weights', W2)
    tf.summary.histogram('bias', B2)
    tf.summary.histogram('act', Y2)
    
    register_conv_layer_inspection(Y2, 14, 'conv2/filters')
    
with tf.name_scope('conv3'): 
    W3 = tf.Variable(tf.truncated_normal([5, 5, L, M], stddev=0.1))
    B3 = tf.Variable(tf.ones([M])/10)
    stride = 2  # output is 7x7
    Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=[1, stride, stride, 1], padding='SAME') + B3)
    
    tf.summary.histogram('weights', W3)
    tf.summary.histogram('bias', B3)
    tf.summary.histogram('act', Y3)
    
    register_conv_layer_inspection(Y3, 7, 'conv2/filters')
    
with tf.name_scope('dense1'): 
    # the output from Y3 is a matrix. The fully connected layer will consume a 1D vector.
    # we must unwrap the matrix into an array so that the W4 weight matrix applies the
    # weights as it should, one for each output from Y3. 
    print Y3
    YY = tf.reshape(Y3, shape=[-1, 7 * 7 * M])
    print YY.get_shape().as_list()
    print YY
    
    W4 = tf.Variable(tf.truncated_normal([7 * 7 * M, N], stddev=0.1))
    
    print W4
    
    B4 = tf.Variable(tf.ones([N])/10)
    Y4 = tf.nn.relu(tf.matmul(YY, W4) + B4)
    
    tf.summary.histogram('weights', W4)
    tf.summary.histogram('bias', B4)
    tf.summary.histogram('act', Y4)

Tensor("conv3_6/Relu:0", shape=(?, 7, 7, 12), dtype=float32)
[None, 588]
Tensor("dense1_6/Reshape:0", shape=(?, 588), dtype=float32)
<tf.Variable 'dense1_6/Variable:0' shape=(588, 200) dtype=float32_ref>


In [4]:
# we'll use the truncated_normal as it is the recomended random to use for the relu 
# activation function
# The model

with tf.name_scope('output'): 
    W5 = tf.Variable(tf.truncated_normal([N, 10], stddev=0.1))
    B5 = tf.Variable(tf.ones([10])/10)
    Ylogits = tf.matmul(Y4, W5) + B5
    
    tf.summary.histogram('weights', W5)
    tf.summary.histogram('bias', B5)
    tf.summary.histogram('act', Ylogits)
    
with tf.name_scope('prediction'): 
    Y = tf.nn.softmax(Ylogits)

In [5]:
# cross-entropy loss function (= -sum(Y_i * log(Yi)) ), normalised for batches of 100  images
# TensorFlow provides the softmax_cross_entropy_with_logits function to avoid numerical stability
# problems with log(0) which is NaN
with tf.name_scope('loss'): 
    cost = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_)
    cost = tf.reduce_mean(cost)*100
    tf.summary.scalar('cross_enthropy', lr)

In [6]:
max_learning_rate = 0.003
min_learning_rate = 0.0001
decay_speed = 20.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations

# learning rate decay
def get_learning_rate(i):
    learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
    return learning_rate

In [7]:
# the optimizer taking the learning rate input which will
# be configured at each step and will minimize the corss entropy function
with tf.name_scope('training'):
    train_step = tf.train.AdamOptimizer(lr).minimize(cost)

In [8]:
# Same as first version:

# % of correct answers found in batch
# based on the Y and Y_ which will be geneated during the runtime of the tests
# we compose this is_correct to compute how right we are during training
with tf.name_scope('accuracy'):
    is_correct = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))
    
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    tf.summary.scalar('accuracy_value', accuracy)

In [9]:
# we start our trainig session
session_name = '10'
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    summary_op = tf.summary.merge_all()
    writer = tf.summary.FileWriter('{}/{}'.format(TPM_TENSORBOARD_PATH, session_name))
    writer.add_graph(sess.graph)
    
    for epoch in range(30):
        
        learning_rate = get_learning_rate(epoch)
        
        print('Epoch', epoch, 'learning rate', learning_rate)
        
        for i in range(int(mnist.train.num_examples/100)):
            # load a btach of images, thanks to mnist
            batch_X, batch_Y = mnist.train.next_batch(100)
            
            train_data = {X:batch_X, Y_:batch_Y, lr: learning_rate, pkeep: 0.75}
            
            # train the network one time
            summary, acc = sess.run([summary_op, train_step], feed_dict=train_data)

            writer.add_summary(summary, epoch * mnist.train.num_examples / 100 + i)
                
        # get the accuracy after training
        a, c = sess.run([accuracy, cost], feed_dict=train_data)
        print('Current training accuracy', a, ' current error', c)
        print('Current testing accuracy', accuracy.eval({X:mnist.test.images, Y_:mnist.test.labels, lr: learning_rate, pkeep: 0.75}))

    a, c = sess.run([accuracy, cost], feed_dict=train_data)
    print('Final training accuracy', a)        
    print('Final testing accuracy', accuracy.eval({X:mnist.test.images, Y_:mnist.test.labels, lr: learning_rate, pkeep: 0.75}))

('Epoch', 0, 'learning rate', 0.003)


KeyboardInterrupt: 